In [1]:
#Example from paper P'Learning Abstracted Non-deterministic Finite State Machines'.
#    https://link.springer.com/chapter/10.1007/978-3-030-64881-7_4

In [2]:
# Import statemets

import random

from aalpy.base import SUL
from aalpy.oracles import RandomWalkEqOracle
from aalpy.learning_algs import run_abstracted_Lstar_ONFSM
from aalpy.SULs import MealySUL
from aalpy.utils import load_automaton_from_file

In [5]:
# Mapper/SUL used for abstracted learning 

class Multi_Client_MQTT_Mapper(SUL):
    def __init__(self):
        super().__init__()

        five_clients_mqtt_mealy = load_automaton_from_file('../DotModels/five_clients_mqtt_abstracted_onfsm.dot',
                                                           automaton_type='mealy')
        self.five_client_mqtt = MealySUL(five_clients_mqtt_mealy)
        self.connected_clients = set()
        self.subscribed_clients = set()

        self.clients = ('c0', 'c1', 'c2', 'c3', 'c4')

    def get_input_alphabet(self):
        return ['connect', 'disconnect', 'subscribe', 'unsubscribe', 'publish']

    def pre(self):
        self.five_client_mqtt.pre()

    def post(self):
        self.five_client_mqtt.post()
        self.connected_clients = set()
        self.subscribed_clients = set()

    def step(self, letter):
        client = random.choice(self.clients)
        inp = client + '_' + letter
        concrete_output = self.five_client_mqtt.step(inp)
        all_out = ''

        if letter == 'connect':
            if client not in self.connected_clients:
                self.connected_clients.add(client)
            elif client in self.connected_clients:
                self.connected_clients.remove(client)
                if client in self.subscribed_clients:
                    self.subscribed_clients.remove(client)
                if len(self.subscribed_clients) == 0:
                    all_out = '_UNSUB_ALL'

        elif letter == 'subscribe' and client in self.connected_clients:
            self.subscribed_clients.add(client)
        elif letter == 'disconnect' and client in self.connected_clients:
            self.connected_clients.remove(client)
            if client in self.subscribed_clients:
                self.subscribed_clients.remove(client)
            if len(self.subscribed_clients) == 0:
                all_out = '_UNSUB_ALL'
        elif letter == 'unsubscribe' and client in self.connected_clients:
            if client in self.subscribed_clients:
                self.subscribed_clients.remove(client)
            if len(self.subscribed_clients) == 0:
                all_out = '_ALL'

        concrete_outputs = concrete_output.split('__')
        abstract_outputs = set([e[3:] for e in concrete_outputs])
        if 'Empty' in abstract_outputs:
            abstract_outputs.remove('Empty')
        if abstract_outputs == {'CONCLOSED'}:
            if len(self.connected_clients) == 0:
                all_out = '_ALL'
            return 'CONCLOSED' + all_out
        else:
            if 'CONCLOSED' in abstract_outputs:
                abstract_outputs.remove('CONCLOSED')
            abstract_outputs = sorted(list(abstract_outputs))
            output = '_'.join(abstract_outputs)
            return '_'.join(set(output.split('_'))) + all_out

In [6]:
# instantiate SUL and input alphabet
sul = Multi_Client_MQTT_Mapper()
alphabet = sul.get_input_alphabet()

# create eq. oracle
eq_oracle = RandomWalkEqOracle(alphabet, sul, num_steps=5000, reset_prob=0.09, reset_after_cex=True)

# define mapping from concrete outputs to abstract outputs
abstraction_mapping = {
    'CONCLOSED': 'CONCLOSED',
    'CONCLOSED_UNSUB_ALL': 'CONCLOSED',
    'CONCLOSED_ALL': 'CONCLOSED',
    'UNSUBACK' : 'UNSUBACK',
    'UNSUBACK_ALL': 'UNSUBACK'
}

In [8]:
# Start learning process

learned_abstracter_onfsm = run_abstracted_Lstar_ONFSM(alphabet, sul, eq_oracle, abstraction_mapping=abstraction_mapping,
                                           n_sampling=200, print_level=2)

Hypothesis 1 has 3 states.
-----------------------------------
Learning Finished.
Learning Rounds:  1
Number of states: 3
Time (in seconds)
  Total                : 13.62
  Learning algorithm   : 13.62
  Conformance checking : 0.0
Learning Algorithm
 # Membership Queries  : 232995
 # Steps               : 985895
Equivalence Query
 # Membership Queries  : 418
 # Steps               : 5000
-----------------------------------


In [9]:
# Print the automaton (note that the original, non-abstracted automataon has 273 states)
print(learned_abstracter_onfsm)

digraph learnedModel {
s0 [label=s0];
s1 [label=s1];
s2 [label=s2];
s0 -> s1  [label="connect/CONNACK"];
s0 -> s0  [label="disconnect/CONCLOSED_ALL"];
s0 -> s0  [label="subscribe/CONCLOSED_ALL"];
s0 -> s0  [label="unsubscribe/CONCLOSED_ALL"];
s0 -> s0  [label="publish/CONCLOSED_ALL"];
s1 -> s0  [label="connect/CONCLOSED_ALL"];
s1 -> s1  [label="connect/CONNACK"];
s1 -> s1  [label="connect/CONCLOSED_UNSUB_ALL"];
s1 -> s0  [label="disconnect/CONCLOSED_ALL"];
s1 -> s1  [label="disconnect/CONCLOSED"];
s1 -> s1  [label="disconnect/CONCLOSED_UNSUB_ALL"];
s1 -> s2  [label="subscribe/SUBACK"];
s1 -> s1  [label="subscribe/CONCLOSED"];
s1 -> s1  [label="unsubscribe/UNSUBACK_ALL"];
s1 -> s1  [label="unsubscribe/CONCLOSED"];
s1 -> s1  [label="publish/PUBACK"];
s1 -> s1  [label="publish/CONCLOSED"];
s2 -> s0  [label="connect/CONCLOSED_ALL"];
s2 -> s2  [label="connect/CONNACK"];
s2 -> s2  [label="connect/CONCLOSED"];
s2 -> s1  [label="connect/CONCLOSED_UNSUB_ALL"];
s2 -> s0  [label="disconnect/CONCL